In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def displaycorr(df0):
    corr= df0.corr()


    # Generate a mask for the upper triangle
    mask = np.ones_like(corr, dtype=bool)
    for i in range(len(mask[0])):
        for j in range(len(mask[1])):
            if j<=i:
                mask[i][j] = not mask[i][j]


    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(10,10))

    # Generate a custom diverging colormap


    #if np.random.randint(10, size=1)[0]  not in (1,0):
    #    color = 'rocket_r'
    #    cmap = sns.color_palette(color, as_cmap=True)
    #elif np.random.randint(2, size=1)[0]:
    color = "Spectral"
    cmap = sns.color_palette(color, as_cmap=True)
    #else:
     #   cmap = sns.diverging_palette(210, 110, l=80, center="light", as_cmap=True)


    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0, annot=True,
                square=True, linewidths=.5, cbar_kws={"shrink": .7})

display_info=False
df=[]
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        if ".csv" in filename:
            df.append(pd.read_csv(f'{dirname}/{filename}'))
features_df=df[1]
temp_df=df[0]

#detecto nulls
if display_info:
    features_df.info()
    print('--Nulls--')
    print(features_df.isna().sum())
    print(features_df.describe().T)
#sacar nulls    
#ESTA MAL SACAR LOS NULLOS DE PAGO MINIMO?----------------------------------------
aux=[]    
id_df = pd.DataFrame(aux, columns=['CUST_ID', 'PHONE', 'CONTACTS'])
id_df[['CUST_ID', 'PHONE', 'CONTACTS']] = temp_df['CUST_ID;PHONE;CONTACTS'].str.split(';', expand=True)
id_df.describe().T
    
features_df.drop(['CUST_ID'], axis=1)
data_frame = pd.concat([features_df.drop(['CUST_ID'], axis=1),id_df],axis=1)
data_frame= data_frame.dropna(axis=0, subset=['MINIMUM_PAYMENTS'])
data_frame= data_frame.dropna(axis=0, subset=['CREDIT_LIMIT'])
data_frame.info()
#correlacion
displaycorr(data_frame)
data_frame.hist(bins=30, figsize=(20, 14))
##---------------------------DATA RREADY TO USE-----------------------  

Parte 3
Utilizando el algoritmo k-means efectúe una clusterización del dataset entregado.
Detecte aquellos clusters relevantes para el problema, describiéndolos en términos de negocio. Debe de graficar el resultado de la clusterización, se recomienda el uso de
PCA para bajar la dimensionalidad de los datos.
Utilice las técnicas que considere necesarias para determinar la cantidad óptima de
clusters, justificando claramente las decisiones tomadas.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd
from sklearn import preprocessing




def plot3d(x,y,z,colors):
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib import cm
    from matplotlib.ticker import LinearLocator, FormatStrFormatter
    from mpl_toolkits.mplot3d import Axes3D

    ## Matplotlib Sample Code using 2D arrays via meshgrid
    threedee = plt.figure().gca(projection='3d')
    threedee.scatter(x, y, z,c=colors, edgecolor='none', alpha=0.51,
            cmap=plt.cm.get_cmap('Spectral',7))
    plt.show()
def best_K_cluster_silueta(x,a,b):
    from sklearn.metrics import silhouette_score 
    from sklearn.cluster import KMeans
    import matplotlib.pyplot as plt
    sil = []
    for k in range(a,b):
        km=KMeans(n_clusters =k).fit(x)
        sil.append(silhouette_score(x,km.labels_,metric='euclidean'))
    plt.plot(range(a,b),sil,'bx-')
    plt.title('Silueta')
    plt.show()
def best_K_cluster_codo(x,b):
    from sklearn.cluster import KMeans
    from scipy.spatial.distance import cdist
    import matplotlib.pyplot as plt
    cod = []
    for k in range(1,b):
        km=KMeans(n_clusters =k).fit(x)
        cod.append(sum(np.min(cdist(x,km.cluster_centers_,'euclidean'),axis=1))/x.shape[0])
    plt.plot(range(1,b),cod,'bx-')
    plt.title('Codo')
    plt.show()
    
    
    
features_df=data_frame.copy().drop(['CUST_ID', 'PHONE', 'CONTACTS'], axis=1) # removemos contactos porque no tinen correlacion enontces queda como un vector en el pca y molesta
features_df.head()


pca = PCA(n_components=17)
pca.fit(features_df)
pca_samples = pca.transform(features_df)#veo las pocisiones reales no noramlizadas
ps = pd.DataFrame(pca_samples)
#displaycorr(ps)
features_df.head()
#plot3d(ps[0], ps[1], ps[2])

fig = plt.figure(figsize=(8,8))
plt.plot(ps[0], ps[1], 'o', markersize=4, color='blue', label='class1')

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()
#plot3d(-ps[0], ps[1], ps[2])
#plot3d(ps[0], -ps[1], ps[2])
#plot3d(-ps[0], -ps[1], ps[2])
#plt.show()
#best_K_cluster_silueta(features_df,2,20)




In [ ]:
#best_K_cluster_codo(features_df,20)
# no normalizado                  
                   
algorithm = (KMeans(n_clusters =6 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
algorithm.fit(features_df)
algorithm.labels_
print( algorithm.cluster_centers_)
algorithm.labels_.shape
print(algorithm.labels_)
plot3d(ps[0], ps[1], ps[2],algorithm.labels_)
plt.scatter(ps[0], ps[1],
            c=algorithm.labels_, edgecolor='none', alpha=0.51,
            cmap=plt.cm.get_cmap('Spectral',7))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();



In [ ]:
#normalizado
x = features_df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
features_df_norm = pd.DataFrame(x_scaled)


pca = PCA(n_components=17)
pca.fit(features_df_norm)
pca_samples = pca.transform(features_df_norm)
psn = pd.DataFrame(pca_samples)
#displaycorr(ps)e
features_df.head()
fig = plt.figure(figsize=(8,8))
plt.plot(psn[0], psn[1], 'o', markersize=4, color='blue', label='class1')

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()
                   
algorithmn = (KMeans(n_clusters =6 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
algorithmn.fit(features_df_norm)
algorithmn.labels_
print( algorithmn.cluster_centers_)
algorithmn.labels_.shape
print(algorithmn.labels_)

plot3d(psn[0], psn[1], psn[2],algorithmn.labels_)

plt.scatter(psn[0], psn[1],
            c=algorithmn.labels_, edgecolor='none', alpha=0.51,
            cmap=plt.cm.get_cmap('Spectral',7))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();




In [ ]:
#extra 
#estas funciones demoran y son cpu consuming sobretodo la silueta
best_K_cluster_silueta(features_df,2,20)
best_K_cluster_silueta(features_df_norm,2,20)
best_K_cluster_codo(features_df,20)
best_K_cluster_codo(features_df_norm,20)

#============K=2 clusters=========
#usar 6 por le segundo maximo relativo de siluera 
                   

In [ ]:
#Los clusters no son iguales
#clusters encontrados con los datos normalizados vistos en el dataframe sin normalizar
plt.scatter(ps[0], ps[1],
            c=algorithmn.labels_, edgecolor='none', alpha=0.51,
            cmap=plt.cm.get_cmap('Spectral',7))
plt.xlabel('vector 1')
plt.ylabel('vector 2')
plt.colorbar();

In [ ]:
#clsuters encontrados con los datos sin normalizar vistos sobre el dataframe normalizado
plt.scatter(psn[0], psn[1],
            c=algorithm.labels_, edgecolor='none', alpha=0.51,
            cmap=plt.cm.get_cmap('Spectral',7))
plt.xlabel('vector 1')
plt.ylabel('vector 2')
plt.colorbar();

Parte 4
________________________

En sus propias palabras, describa las técnicas de clustering basadas en densidad, en
particular el algoritmo DBSCAN. Aplique DBSCAN al dataset y compare los resultados
con los obtenidos en la parte 3.

--------------------
Al clusterizar basado en densidad losn puntos se separan en 2 tipos, los que pertenecen a una agrupacion de puntos y los puntos de 'ruido'.El primer cluster esta formado por los puntos de ruidos  mientras el resto de los clusters se definen por los puntos que estan proximos.En el otro tipo de clusterizacion los puntos de ruidos se agruparian a los clusters más cercanos y no a el ruido,ademas no estan agrupados por un centroide por lo que los clusters se ven mas 'naturales'.




In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import preprocessing
from collections import Counter


density_clustering = DBSCAN(eps=0.3, min_samples=4).fit(features_df_norm)    #minsampoles tinee que ser mayor a uno sno cada punto es un cluster minimo
print(density_clustering.labels_)
plt.scatter(psn[0], psn[1],
            c=density_clustering.labels_, edgecolor='none', alpha=0.51,
            cmap=plt.cm.get_cmap('Spectral',4))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar()
plt.show


plot3d(psn[0], psn[1], psn[2],density_clustering.labels_)

In [ ]:
plt.scatter(ps[0], ps[1],
            c=density_clustering.labels_, edgecolor='none', alpha=0.51,
            cmap=plt.cm.get_cmap('Spectral',4))
plt.xlabel('component 1')
plt.ylabel('component 2')
print(density_clustering.labels_)
density_clustering

plot3d(ps[0], ps[1], ps[2],density_clustering.labels_)
print(Counter(density_clustering.labels_)) #ver la cantidad de clusters


Parte 5
Estudie y explique el método de Ward utilizado en clustering aglomerativo, en
particular, la implementación de sklearn.

El clustering aglomerativo del agrupamiento jerárquico es un método de análisis de grupos
que agrupa los elementos desde las hojas hacia la raiz del árbol, cada observación 
comienza en su propio grupo (en las hojas del árbol), y los elementos de los grupos son 
mezclados mientras uno sube en la jerarquia añadiendo elementos que tienen un mismo 
padre en común.

Un cluster aglomerativo tiene un comportamiento en el que "el rico se vuelve más rico", 
provocando clusters desiguales y agrupando nuevos elementos en los clusters de mayor tamaño.

Especificamente el Ward, minimiza la suma cuadrada de diferencias entre todos los clusters. 
Presentando un acercamiento donde se minimiza la varianza y distancia entre los elementos, 
de cierta forma teniendo un funcionamiento similar al de k-means pero abordado con 
jerarquias aglomerativas. Por su funcionamiento, este permite tener clusters mejor 
balanceados.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
cluster  = AgglomerativeClustering(6,linkage='ward').fit(features_df_norm) 

In [ ]:
print(cluster.labels_)
plt.scatter(ps[0], ps[1],
            c=cluster.labels_, edgecolor='none', alpha=0.51,
            cmap=plt.cm.get_cmap('Spectral',4))
plt.xlabel('component 1')
plt.ylabel('component 2')

plot3d(ps[0], ps[1], ps[2],cluster.labels_)
print(Counter(cluster.labels_)) #ver la cantidad de clusters



Parte 6
Planificando próximas etapas, el banco requiere armar un cuestionario para nuevos
clientes, de forma de poder ofrecer el producto a usuarios de los cuales no cuenta con
histórico transaccional. Para esto, el Departamento Comercial solicita que se
construya, a partir del análisis de datos, el listado de preguntas y un modelo predictivo
que en función de las respuestas retorne la propensión de compra de los productos.
En caso de que entienda que no es posible cumplir con alguno de estos
requerimientos, documente los motivos que lo llevaron a tomar esa conclusión.

Tomariamos la decisiones bandasnos en un cuestionarios booleano donde cada si sumaria puntos ,a la hora ofrecer los servicios nos fijariamos si puntaje del cuestionario es mayor a 60% de #preguntas

Las preguntas serian del estilo

Va a depositar mas de X dolares en su cuenta de banco? (x siendo 0.6 del  maximo actual de balance)
Cuenta con un ingreso liquido superior a Y dolares por mes?(especular el PRC_FULL_PAYMENT)
Frecuenta sitios de compra como Amazon, MercadoLibre ,TiendaMia?()
Viaja al menos una vez cada 3 meses?(porpenso a nnececitar una tarjeta internacional)
Suele realizar compras en productos no esenciales con un valor superior a Z dolares?


In [ ]:
############# no correr### 128 plots
for i in range(17):
    for j in range(17):
        if j>i and i>0:
            plot3d(psn[i], psn[j],psn[0],algorithmn.labels_)

In [ ]:
#estudipo de a quiens ofrecerles los servicios
data = algorithmn.cluster_centers_
columns=features_df.columns
final_df = pd.DataFrame(data,  columns=columns)
final_df.head(7)

Parte 7
Ensaye una discusión general del trabajo realizado, haciendo los comentarios y
recomendaciones que considere necesarias.

Outliners: la eliminzacion de outliner no la vimos necesaria, nos guiamos por el poensamiento de 'porque una persona tenga mucha o poca plata no significa que yo como banco no voy a ofrecerle mi servicios',este pensamiento está condicionado por idea de que como banco no pierdo nada ofreciendo a todos mis clientes un servicio. 

Los nullos decidimos eliminarlos del dataset porque no sabesmos si hacen referenica a ser 0 o a que flata informcaion por lo que lo tomamos como que falta.

A la hora de elegir la cantidad de clusters para el kmeans al principio usamos 2 pero decidimos que estos no eran representativos para  los "tipos" de clientes que tiene el banco por lo que usamos una segunda opcion de 6 clusters.

Otro dilema fue el de normalizar los datos o no(para el kmeans) asi que probamos los 2 caminos y comparamso que no era lo mismo. Concluimos que ningun feature era sustancialmente más importante sobre los otros al decidir para los 2 sevicios por lo que usamos los datos normalizados(en el caso de cluster por densidad era necesarioq ue fuera normalizado)
Para la decision de a quienes ofrecer lso servicios usamos los centroides de los clusters normalizados.Los centroides representarian un cliente tipo , entonces el problema se reduce a elegir si ofrecer servicios a 6 'clientes' o no.






Despues de normalizar el dataset notamos que y ver su representacion con PCA (en los valores mas importantes(con mayor valor propio asociado)) vimos que los puntos estaban bastantes concentrados, asi que supusimos que los algoritmos de clustrizacion por densidad iban a ser poco efectivos(luego se visualizamos esto al usar DBSCAN)   


Tarjeta Platinum
-----------------------
En primera instancia a los clientes del cluster 5 y 3 no seria recomendable ofrecer este servicio , por el bajo balance en sus cuentas(sobre todo en el 3) y (en el 5) por el poco uso de esta(BALANCE_FREQUENCY  PURCHASES PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,CASH_ADVANCE_FREQUENCY	CASH_ADVANCE_TRX),
Los clientes de los clusters 4 y 1 son los seria mas benefectorio ofrecer el servicio
La decision de ofrecercelo a los clientes del cluster 1 y 2 es conflictiva ya que tiene  una baja freceuncia de pago y porcentaje de pago. 
Internet Global
-----------------------
Posibles candidatos para este servicio serian el 1,3 y 4 debido a su alto PURCHASES_INSTALLMENTS_FREQUENCY y  INSTALLMENTS_PURCHASES sobre todo 1 y 4 por su alto BALANCE
Mientras los clientes del cluster 2 y 5 tiene  bajo INSTALLMENTS_PURCHASE y PURCHASES_FREQUENCY	
El 0 es controvercial tiene alto PURCHASES_FREQUENCY pero bajo  INSTALLMENTS_PURCHASE , BALANCE medio  alto ONEOFF_PURCHASES_FREQUENCY, es un estereotipo de cliente promedio en varios features, personalmente le ofreceriamos ambos de  servicios